In [1]:
%load_ext autoreload
%autoreload 2
import SpectralConvxD as spc
import numpy as np
import tensorflow as tf
import copy 



In [2]:
x_train, y_train,x_test, y_test=spc.generate_data(name_data='mnist1d')


Did or could not load data from ./mnist1d_data.pkl. Rebuilding dataset...


In [3]:
maxpooling_config ={'strides': 1,
                     'padding': 'valid' }

hyperparameters = { 'filters' :20,
                    'input_shape' :(40,),
                    'learning_rate' : 0.03,
                    'epochs' : 10,
                    'batch_size' :100,
                    'activation': 'relu',
                    'labels' : 10,
                    'conxd':1,
                    'pool_size':2,
                    'use_pruning':False,
                    'pre_training': False,
                    'post_training':True
                    }

In [4]:
replication = {'rep':1} 
depth={'N':[2500]}
drop={'p':[0,0.05,0.1,0.15,0.2,0.25, 0.3,0.35, 0.4,0.45, 0.5,0.55, 0.6,0.65, 0.7, 0.75,0.8, 0.85,0.9,0.95,1]}
path =r"C:\Users\jketchak\Documents\DALPHAcommunity\UNAMUR\EUREKA\btwg_ml\SpectralConvxD\SpectralConvxD\DataWarehouse\dtest"
path=spc.utils.remplacer_backslash(path)

In [7]:
inv=2
models_pars,models_name= spc.pars()
models = spc.SpectralCnn(hyperparameters=hyperparameters,maxpooling_config=maxpooling_config)
models.compile_models(units=2500,spectral_config=models_pars.get(models_name[inv]).get('spectral_config'),
                      spectral_cnn1d_config=models_pars.get(models_name[inv]).get('spectral_cnn1d_config'),
                      spectral_cnn2d_config=models_pars.get(models_name[inv]).get('spectral_cnn2d_config'),
                      name=models_name[inv],
                      layers_name=['convx','spec1','spec2'],
                      trainable_weights=None,
                      layer_cible=None)
models.summary(name=models_name[inv])

Pruning: Pershaps pre-training
Post training: Spectral compactness
No trainable weights provided, using default initialization
specConvXd model summary:
 


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ convx (SpecCnn1D)               │ (None, 40, 20)         │           920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 39, 20)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 780)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spec1 (Spectral)                │ (None, 2500)           │     1,952,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spec2 (Spectral)                │ (None, 10)             │        25,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,978,430 (7.55 MB)

 Trainable params: 1,978,430 (7.55 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
models.train(x_train, y_train, x_test, y_test, name=models_name[inv],verbose=1)

Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.1972 - loss: 2.0633 - val_accuracy: 0.4525 - val_loss: 1.3281
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4871 - loss: 1.2451 - val_accuracy: 0.5560 - val_loss: 1.0373
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6105 - loss: 0.9682 - val_accuracy: 0.6780 - val_loss: 0.8221
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7252 - loss: 0.7103 - val_accuracy: 0.7220 - val_loss: 0.7157
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7943 - loss: 0.5386 - val_accuracy: 0.7720 - val_loss: 0.5778
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8506 - loss: 0.3967 - val_accuracy: 0.8020 - val_loss: 0.5057
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9017 - loss: 0.2812 - val_accuracy: 0.8185 - val_loss: 0.4680
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9238 - loss: 0.2201 - val_accuracy: 0.8535 - val_loss:

In [9]:
for var in models.model.get_layer(name='spec1').variables:
    print(var.name, var.shape, var.trainable)

base (780, 2500) True
bias (2500,) True


In [ ]:
pre_taining_weights=models.get_trainable_weights(layer_name='spec1')
pre_taining_weights

In [ ]:
inv=2
models = spc.SpectralCnn(hyperparameters=hyperparameters,maxpooling_config=maxpooling_config)
models.compile_models(units=2500,spectral_config=models_pars.get(models_name[inv]).get('spectral_config'),
                      spectral_cnn1d_config=models_pars.get(models_name[inv]).get('spectral_cnn1d_config'),
                      spectral_cnn2d_config=models_pars.get(models_name[inv]).get('spectral_cnn2d_config'),
                      name=models_name[inv],
                      layers_name=['convx','spec1','spec2'],
                      trainable_weights=pre_taining_weights,
                      layer_cible='spec1')
models.summary(name=models_name[inv])

In [ ]:
models.evaluate(x_test, y_test, trainable_weights=None, path=path, name=models_name[inv], order=0,layer_name='spec1',p=0.0)

In [ ]:
models_name[inv]

In [ ]:
models.model.get_layer(name='spec1').set_weights(tw_copy)

## Pruning model 

In [ ]:
models.cut_value

In [ ]:
for p in drop.get('p'):
    models.evaluate(x_test, y_test, name=models_name[inv],layers=[3],p=p,order=0,path=path)

In [ ]:
results={name : spc.extract_accuracy_and_save(replication=replication,depth=depth,drop=drop,name=name,path=path) for name in models_name if name not in ['Dspec']}


In [ ]:
spc.plot_results(results,x_min=0,
            x_max=1,
            y_min=0.2,
            y_max=1.0,
            xlabel='p',
            ylabel='accuracy',
            show_fig=True,
            use_grid=False,
            markersize=3,
            percentile=True,
            save_fig=False,
            dpi=300,
            loc='lower right',
            name_fig='results')

## Pre-training

In [ ]:
models_pars.get('specConvXd').get('spectral_cnn1d_config')['trainable_phi']=True
models_pars.get('specConvXd').get('spectral_cnn1d_config')['use_lambda_in']=False

In [ ]:
inv=2
hyperparameters['use_pruning']=False
models_pars.get('specConvXd').get('spectral_cnn1d_config')['trainable_phi']=True
models_pars.get('specConvXd').get('spectral_cnn1d_config')['use_lambda_in']=False
models = spc.SpectralCnn(hyperparameters=hyperparameters,maxpooling_config=maxpooling_config)
models.compile_models(units=2500,spectral_config=models_pars.get(models_name[inv]).get('spectral_config'),
                      spectral_cnn1d_config=models_pars.get(models_name[inv]).get('spectral_cnn1d_config'),
                      spectral_cnn2d_config=models_pars.get(models_name[inv]).get('spectral_cnn2d_config'),
                      name=models_name[inv])
models.summary(name=models_name[inv])
models.train(x_train, y_train, x_test, y_test, name=models_name[inv],verbose=1,layers=[3])

In [ ]:
models.weigths[0][0].shape